<a href="https://colab.research.google.com/github/bala1802/ERA_Session17/blob/main/S17_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchinfo --quiet
!pip install transformers --quiet

In [2]:
cd /content/drive/MyDrive/Session-17

/content/drive/MyDrive/Session-17


In [3]:
from transformers import AutoTokenizer
import torch
import transformer
from torchinfo import summary
import numpy as np
from tqdm.auto import tqdm

In [4]:
n_iterations = 20
batch_size   = 16
seq_len      = 64 # also known as blocks in gpt
eval_iteration = 5
total_iterations_for_evaluation = 10
device       = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
data_pth = 'data/english.txt'
data_raw = open(data_pth, encoding ="utf-8").read()

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
vocab_size

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

30522

In [9]:
tokens = tokenizer.tokenize(data_raw)
token_indices = tokenizer.convert_tokens_to_ids(tokens)
data_tokens = torch.tensor(token_indices, dtype = torch.long)
len(data_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (37443 > 512). Running this sequence through the model will result in indexing errors


37443

In [10]:
data_tokens

tensor([ 5219,  1014,  1011,  ..., 12375,  2015,  1012])

In [12]:
n = int(0.9 * len(data_tokens))
train_data = data_tokens[:n]
val_data = data_tokens[n:]

In [13]:
my_gpt = transformer.Gpt(n_embeddings = vocab_size)
my_gpt.to(device)
optimizer = torch.optim.Adam(params = my_gpt.parameters())
summary(model= my_gpt, input_size=(32,64), dtypes = [torch.int32],col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
Gpt (Gpt)                                                    [32, 64]             [32, 64, 30522]      49,152               True
├─Embedding (embeddings)                                     [32, 64]             [32, 64, 768]        23,440,896           True
├─Dropout (embedding_dropout)                                [32, 64, 768]        [32, 64, 768]        --                   --
├─Sequential (transformer_decoder)                           [32, 64, 768]        [32, 64, 768]        --                   True
│    └─TransformerDecoderBlock (0)                           [32, 64, 768]        [32, 64, 768]        --                   True
│    │    └─MultiHeadAttentionBlock_Decoder (msa_block)      [32, 64, 768]        [32, 64, 768]        2,360,064            True
│    │    └─MLPBlock (mlp_block)                             [32, 64, 768]        [32, 64, 768

In [14]:
def get_batch(data: list[str],seq_len:int, batch_size: int):
  ix = torch.randint(len(data) - seq_len, (batch_size,))
  x = torch.stack([data[i: i + seq_len] for i in ix])
  y = torch.stack([data[i+1 : i + seq_len + 1] for i in ix])
  return x,y

In [16]:
train_loss=0

In [17]:
for it in tqdm(range(n_iterations)):
  my_gpt.train()
  x,y = get_batch(data = train_data,seq_len = seq_len, batch_size = batch_size)
  x = x.to(device)
  y = y.to(device)

  logits,loss   = my_gpt(x,y)
  train_loss += loss.item()

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  # evaluation
  if it % eval_iteration == 0:
      print("Evaluating the model")
      my_gpt.eval()
      losses = torch.zeros(total_iterations_for_evaluation)
      for k in range(total_iterations_for_evaluation):
          x,y = get_batch(data=val_data, seq_len = seq_len, batch_size=batch_size)
          x = x.to(device)
          y = y.to(device)
          logits, loss = my_gpt(x, y)
          losses[k] = loss.item()
      val_loss = losses.mean()

      losses = torch.zeros(total_iterations_for_evaluation)
      for k in range(total_iterations_for_evaluation):
          x,y = get_batch(data=train_data, seq_len = seq_len, batch_size=batch_size)
          x = x.to(device)
          y = y.to(device)
          logits, loss = my_gpt(x, y)
          losses[k] = loss.item()
      train_loss = losses.mean()
      print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(it, train_loss, val_loss))

  0%|          | 0/20 [00:00<?, ?it/s]

Evaluating the model
step          0 | train loss 9.3990 | val loss 9.4773
Evaluating the model
step          5 | train loss 6.9147 | val loss 7.3087
Evaluating the model
step         10 | train loss 6.7451 | val loss 7.2380
Evaluating the model
step         15 | train loss 6.6627 | val loss 7.0864
